## Data Cleanup

Before the CSE-CIC-IDS2018 dataset can be used for analysis and training the dataset has to be cleaned. In its raw format the dataset consists of ten individual csv files, each containing the recorded network traffic of a single day of operation, named after the day the traffic was recorded on.

To conduct an initial analysis of the dataset a single file is loaded and dissected.

In [1]:
# set base path to the directory containing the csv files of the dataset
dataset_base_path = r'/path/to/dataset'

### 1. Removing invalid rows

In [2]:
import numpy as np
import pandas as pd
import os

file_path = os.path.join(dataset_base_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')

df = pd.read_csv(file_path)

/anaconda3/envs/spark/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Analyzing the output of `read_csv` shows that for most of the columns pandas could not infer a datatype.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331125 entries, 0 to 331124
Data columns (total 80 columns):
Dst Port             331125 non-null object
Protocol             331125 non-null object
Timestamp            331125 non-null object
Flow Duration        331125 non-null object
Tot Fwd Pkts         331125 non-null object
Tot Bwd Pkts         331125 non-null object
TotLen Fwd Pkts      331125 non-null object
TotLen Bwd Pkts      331125 non-null object
Fwd Pkt Len Max      331125 non-null object
Fwd Pkt Len Min      331125 non-null object
Fwd Pkt Len Mean     331125 non-null object
Fwd Pkt Len Std      331125 non-null object
Bwd Pkt Len Max      331125 non-null object
Bwd Pkt Len Min      331125 non-null object
Bwd Pkt Len Mean     331125 non-null object
Bwd Pkt Len Std      331125 non-null object
Flow Byts/s          329291 non-null object
Flow Pkts/s          331125 non-null object
Flow IAT Mean        331125 non-null object
Flow IAT Std         331125 non-null object
Flow IAT 

Querying the `info()` method of the dataframe shows that pandas inferred all columns as `object` columns as opposed to  numerical columns which would be appropriate for most of them.
In order to understand why the columns are interpreted as `object`s low cardinality columns are analyzed to show individual values.

In [4]:
df['Protocol'].value_counts()

6           170066
17           95674
6            42833
17           15378
0             4596
0             2553
Protocol        25
Name: Protocol, dtype: int64

In [5]:
df['FIN Flag Cnt'].value_counts()

0               268629
0                60520
1                 1707
1                  244
FIN Flag Cnt        25
Name: FIN Flag Cnt, dtype: int64

The unique values indicate the existence of the column name as values in the dataset. 
A visual examination of the input file confirms that the headers are present multiple times within the file, interweaved with the raw data rows. This suggests that a single file was created by concatenating mulitple csv files duplicating the headers in the process.
To fix this issue all columns containing the headers are removed from the dataframe.

In [6]:
df = df[~df['Dst Port'].str.contains('Dst Port', na=False)]

In the next steps the dataframe is exported to a temporary csv file in order to read it again with the correct column datatypes. 
Furthermore the column names are converted to lowercase with non-word characters removed for easier access of the columns.

In [8]:
import re

tmp_path = os.path.join(dataset_base_path, 'tmp')

if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

column_name_regex = re.compile(r"\W", re.IGNORECASE)
df.columns = [column_name_regex.sub('_', c.lower()) for c in df.columns]

tmp_file_path = os.path.join(tmp_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter_duplicate_headers_removed.csv')

df.to_csv(tmp_file_path)

### 2. Removing invalid values

Now the temporary file is loaded with the following datatype definitions. 

In [9]:
types = {
    'dst_port': 'uint32',
    'protocol': 'uint8',
    'timestamp': 'object',
    'flow_duration': 'int64',
    'tot_fwd_pkts': 'uint32',
    'tot_bwd_pkts': 'uint32',
    'totlen_fwd_pkts': 'uint32',
    'totlen_bwd_pkts': 'uint32',
    'fwd_pkt_len_max': 'uint16',
    'fwd_pkt_len_min': 'uint16',
    'fwd_pkt_len_mean': 'float32',
    'fwd_pkt_len_std': 'float32',
    'bwd_pkt_len_max': 'uint16',
    'bwd_pkt_len_min': 'uint16',
    'bwd_pkt_len_mean': 'float32',
    'bwd_pkt_len_std': 'float32',
    'flow_byts_s': 'float64',
    'flow_pkts_s': 'float64',
    'flow_iat_mean': 'float32',
    'flow_iat_std': 'float32',
    'flow_iat_max': 'int64',
    'flow_iat_min': 'int64',
    'fwd_iat_tot': 'int64',
    'fwd_iat_mean': 'float32',
    'fwd_iat_std': 'float32',
    'fwd_iat_max': 'int64',
    'fwd_iat_min': 'int64',
    'bwd_iat_tot': 'uint32',
    'bwd_iat_mean': 'float32',
    'bwd_iat_std': 'float32',
    'bwd_iat_max': 'uint32',
    'bwd_iat_min': 'uint32',
    'fwd_psh_flags': 'uint8',
    'bwd_psh_flags': 'uint8',
    'fwd_urg_flags': 'uint8',
    'bwd_urg_flags': 'uint8',
    'fwd_header_len': 'uint32',
    'bwd_header_len': 'uint32',
    'flow_byts_s': 'float32',
    'bwd_pkts_s': 'float32',
    'pkt_len_min': 'uint16',
    'pkt_len_max': 'uint16',
    'pkt_len_mean': 'float32',
    'pkt_len_std': 'float32',
    'pkt_len_var': 'float32',
    'fin_flag_cnt': 'uint8',
    'syn_flag_cnt': 'uint8',
    'rst_flag_cnt': 'uint8',
    'psh_flag_cnt': 'uint8',
    'ack_flag_cnt': 'uint8',
    'urg_flag_cnt': 'uint8',
    'cwe_flag_count': 'uint8',
    'ece_flag_cnt': 'uint8',
    'down_up_ratio': 'uint16',
    'pkt_size_avg': 'float32',
    'fwd_seg_size_avg': 'float32',
    'bwd_seg_size_avg': 'float32',
    'fwd_byts_b_avg': 'uint8',
    'fwd_pkts_b_avg': 'uint8',
    'fwd_blk_rate_avg': 'uint8',
    'bwd_byts_b_avg': 'uint8',
    'bwd_pkts_b_avg': 'uint8',
    'bwd_blk_rate_avg': 'uint8',
    'subflow_fwd_pkts': 'uint32',
    'subflow_fwd_byts': 'uint32',
    'subflow_bwd_pkts': 'uint32',
    'subflow_bwd_byts': 'uint32',
    'init_fwd_win_byts': 'int32',
    'init_bwd_win_byts': 'int32',
    'fwd_act_data_pkts': 'uint32',
    'fwd_seg_size_min': 'uint8',
    'active_mean': 'float32',
    'active_std': 'float32',
    'active_max': 'uint32',
    'active_min': 'uint32',
    'idle_mean': 'float32',
    'idle_std': 'float32',
    'idle_max': 'uint64',
    'idle_min': 'uint64',
    'label': 'category'
}

In [10]:
df = pd.read_csv(tmp_file_path, dtype=types)

ValueError: cannot safely convert passed user dtype of float32 for object dtyped data in column 17

The error indicates that column 17 (Flow Byts/s) cannot be parsed. Another visual examination of the file reveals the existence of the string `Infinity` in multiple rows of this column.
The `read_csv()` method of pandas is not able to correctly parse this value as it only recognizes the strings `inf/-inf` as a valid representation of infinity.

To fix this problem all occurrences of `Infinity` are replaced by the string `inf`.

In [11]:
df = pd.read_csv(tmp_file_path)

df_infinity_fixed = df.replace('Infinity', 'inf')

tmp_file_path_inf = os.path.join(tmp_path, 'Thursday-01-03-2018_TrafficForML_CICFlowMeter_infinity_fixed.csv')

df_infinity_fixed.to_csv(tmp_file_path_inf)

/anaconda3/envs/spark/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


After fixing the infinity values the file can successfully be loaded with the given datatypes.

In [12]:
df = pd.read_csv(tmp_file_path_inf, dtype=types)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331100 entries, 0 to 331099
Data columns (total 82 columns):
Unnamed: 0           331100 non-null int64
Unnamed: 0.1         331100 non-null int64
dst_port             331100 non-null uint32
protocol             331100 non-null uint8
timestamp            331100 non-null object
flow_duration        331100 non-null int64
tot_fwd_pkts         331100 non-null uint32
tot_bwd_pkts         331100 non-null uint32
totlen_fwd_pkts      331100 non-null uint32
totlen_bwd_pkts      331100 non-null uint32
fwd_pkt_len_max      331100 non-null uint16
fwd_pkt_len_min      331100 non-null uint16
fwd_pkt_len_mean     331100 non-null float32
fwd_pkt_len_std      331100 non-null float32
bwd_pkt_len_max      331100 non-null uint16
bwd_pkt_len_min      331100 non-null uint16
bwd_pkt_len_mean     331100 non-null float32
bwd_pkt_len_std      331100 non-null float32
flow_byts_s          329266 non-null float32
flow_pkts_s          331100 non-null float64
flow_ia

In [14]:
print(f"Infinity values of flow_byts_s: {df[df['flow_byts_s'] == np.inf]['dst_port'].count()}")
print(f"Null values of flow_byts_s: {df[df['flow_byts_s'].isnull()]['dst_port'].count()}")

Infinity values of flow_byts_s: 1085
Null values of flow_byts_s: 1834


### 3. Cleanup Script

In summary the following clean-up steps must be applied to all files of the dataset:
1. Removal of duplicate headers contained as rows of the dataset.
2. Substitution of occurrences of `Infinity` with `inf`
3. Renaming the column names to remove whitespaces and non-word characters

The following script processes all files of the dataset and stores the output files using a name describing the attack types of the flows contained in the files rather than the date of the flows.

Remark: The file `Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv` contains four columns (`Flow ID`, `Src IP`, `Dst IP`, `Src Port`) not present in any of the other files. As those columns are not required they are dropped upon loading the file.

In [17]:
import numpy as np
import pandas as pd
import os
import re

csv_files = {
 'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv': 'infiltration_28-02-2018.csv',
 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv': 'infiltration_01-03-2018.csv',
 'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv': 'bot_02-03-2018.csv',
 'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-web-xss_sql-injection_22-02-2018.csv',
 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv': 'dos-goldeneye-slowloris_15-02-2018.csv',
 'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv': 'ddos-loic-http-loic-udp_20-02-2018.csv',
 'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv': 'ddos-loic-udp_hoic_21-02-2018.csv',
 'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-ftp-ssh_14-02-2018.csv',
 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv': 'dos-slowhttp-hulk_16-02-2018.csv',
 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv': 'bruteforce-web-xss_sql-injection_23-02-2018.csv'
}

column_name_regex = re.compile(r"\W", re.IGNORECASE)
processed_dir = 'processed'
processed_path = os.path.join(dataset_base_path, processed_dir)

def remove_headers(f):    
    return f[~f['Dst Port'].str.contains('Dst Port', na=False)]

def replace_infinity(f):
    return f.replace('Infinity', 'inf', inplace=True)

def remove_non_word_chars_from_column_names(f):
    return [column_name_regex.sub('_', c.lower()) for c in df.columns]
    
if not os.path.exists(processed_path):
    os.mkdir(processed_path)    
    
for f, out in csv_files.items():
    file_path = os.path.join(dataset_base_path, f)
    output_path = os.path.join(dataset_base_path, processed_dir, out)
    
    df = pd.read_csv(file_path, dtype=str).drop(columns=['Flow ID', 'Src IP', 'Dst IP', 'Src Port'], errors='ignore')
    df = remove_headers(df)
    replace_infinity(df)
    df.columns = remove_non_word_chars_from_column_names(df)
    df.to_csv(output_path, index=False)